In [17]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import statsmodels.api as sm
from datetime import timedelta, datetime

In [2]:
import_directory = 'C:\\Users\\emmah\\Google Drive\\Job Applications\\TechnicalAssessment\\Shift\\data'

df_import =pd.read_csv(import_directory + '\\sample.csv')

C:\Users\emmah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_import.columns

Index(['index_pandas', 'vin', 'days_on_market', 'listing_date_begin',
       'listing_date_end', 'year', 'make', 'model', 'trim', 'body_style',
       'transmission', 'mileage', 'accident_count', 'fuel_economy_city',
       'fuel_economy_highway', 'exterior_color', 'msrp', 'is_cpo',
       'seller_city', 'seller_state', 'seller_type',
       'seller_is_franchise_dealer', 'seller_is_online_only',
       'seller_ships_nationwide', 'list_price', 'shift_buyer_region'],
      dtype='object')

In [4]:
# Drop invalid row that are missing vin or list_price
# Drop variables that are not vehicle attributes
# Drop variables that cannot be used in price predication at the time of listing
drop_cols = ['index_pandas', 
             'days_on_market', 
             'listing_date_end', 'shift_buyer_region']
df_valid = df_import.dropna(subset=['vin', 'list_price']).drop(columns = drop_cols).drop_duplicates().astype({'year': 'str'})
print("Columns: n/{}".format(df_valid.columns))
print("Row count: {}".format(len(df_valid)))
df_valid.sort_values(["vin", "listing_date_begin"])

Columns: n/Index(['vin', 'listing_date_begin', 'year', 'make', 'model', 'trim',
       'body_style', 'transmission', 'mileage', 'accident_count',
       'fuel_economy_city', 'fuel_economy_highway', 'exterior_color', 'msrp',
       'is_cpo', 'seller_city', 'seller_state', 'seller_type',
       'seller_is_franchise_dealer', 'seller_is_online_only',
       'seller_ships_nationwide', 'list_price'],
      dtype='object')
Row count: 395450


,vin,listing_date_begin,year,make,model,trim,body_style,transmission,mileage,accident_count,...,exterior_color,msrp,is_cpo,seller_city,seller_state,seller_type,seller_is_franchise_dealer,seller_is_online_only,seller_ships_nationwide,list_price
57683,0HGCV3F93LA014514,2021-02-23,2020,Honda,Accord,NaN,Sedan,Continuously Variable Transmission,7318.0,NaN,...,Blue,NaN,NaN,"Arlington, TX",TX,DEALER,True,NaN,NaN,26595.0
8404,19XFC1F30LE000679,2021-03-28,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,24772.0,0.0,...,Molten Lava Pearl,NaN,True,"North Richland Hills, TX",TX,DEALER,True,NaN,NaN,21795.0
4298,19XFC1F30LE001184,2021-01-11,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,21846.0,0.0,...,Platinum White Pearl,NaN,True,"Albuquerque, NM",NM,DEALER,True,NaN,NaN,20959.0
4299,19XFC1F30LE001184,2021-01-11,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,21846.0,0.0,...,Platinum White Pearl,NaN,True,"Albuquerque, NM",NM,DEALER,True,NaN,NaN,21505.0
6629,19XFC1F30LE001699,2021-04-17,2020,Honda,Civic,EX Sedan FWD,Sedan,Continuously Variable Transmission,26203.0,0.0,...,Silver,NaN,NaN,"Newport News, VA",VA,DEALER,True,NaN,NaN,21495.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675901,WZ1DB4C0XLW032522,2020-09-05,2020,Toyota,Supra,3.0 Premium RWD,Coupe,Automatic,2880.0,0.0,...,RENAISSANCE RED 2.0,58478.0,NaN,"Corpus Christi, TX",TX,DEALER,True,NaN,NaN,52466.0
666929,WZ1DB4C0XLW032570,2021-04-09,2020,Toyota,Supra,3.0 RWD,Coupe,Automatic,8917.0,0.0,...,Red,53555.0,NaN,"Montclair, CA",CA,DEALER,NaN,NaN,NaN,53555.0
666930,WZ1DB4C0XLW032570,2021-04-09,2020,Toyota,Supra,3.0 RWD,Coupe,Automatic,8917.0,0.0,...,Red,54555.0,NaN,"Montclair, CA",CA,DEALER,NaN,NaN,NaN,54555.0
565595,WZ1DB4C0XLW032858,2020-08-26,2020,Toyota,Supra,3.0 RWD,Coupe,8-Speed Automatic,4689.0,0.0,...,Renaissance Red 2.0,NaN,NaN,"Downingtown, PA",PA,DEALER,NaN,NaN,NaN,52990.0


In [5]:
# Calculate data values for imputation

imputation_dict = {}

# Get median values for imputation of numeric variables
numeric_cols = ['mileage', 'accident_count',
       'fuel_economy_city', 'fuel_economy_highway', 'msrp']
for col in numeric_cols:
    median = df_valid[col].median()
    imputation_dict[col] = median
    
# Get imputation values for boolean variables
boolean_cols = ['is_cpo', 'seller_is_franchise_dealer', 'seller_is_online_only', 'seller_ships_nationwide']
boolean_dict = {}
for col in boolean_cols:
    if len(df_valid[df_valid[col]==True])==0:
        imputation_dict[col] = True
    if len(df_valid[df_valid[col]==False])==0:
        imputation_dict[col] = False
    else:
        pass

# Get imputation for categorical variables with majority categies with subcategory counts < 0.1%
categorical_cols = ["trim", "transmission", "exterior_color"]
for col in categorical_cols:
    imputation_dict[col] = 'Other'

print(imputation_dict)

{'mileage': 11473.0, 'accident_count': 0.0, 'fuel_economy_city': 24.0, 'fuel_economy_highway': 31.0, 'msrp': 35495.0, 'is_cpo': False, 'seller_is_franchise_dealer': False, 'seller_is_online_only': True, 'seller_ships_nationwide': False, 'trim': 'Other', 'transmission': 'Other', 'exterior_color': 'Other'}


In [6]:
# Split into train and test
unique_vins = df_valid['vin'].unique()
print("Number of unique vins: {}".format(len(unique_vins)))

train_vin = np.random.choice(unique_vins, size=int(len(unique_vins)*0.8), replace=False, p=None)
test_vin = unique_vins[~np.in1d(unique_vins, train_vin)]

df_train = df_valid[df_valid['vin'].isin(train_vin)]
df_test = df_valid[df_valid['vin'].isin(test_vin)]
print("test row percentage: {}%".format(round(100*(len(df_test)/len(df_valid)))))

# Check takes long to run
vin_duplicated = []
for i in df_test.index:
    if df_test.loc[i, 'vin'] in list(df_train['vin']):
        vin_duplicated.append(df_test.loc[i, 'vin'])

assert len(vin_duplicated)==0, "Vin number in test and train"

Number of unique vins: 283993
test row percentage: 20%


In [7]:
# Group by observations to summarize 

def summarize_vin(group1):
    """Summarizes observations using mode for categorical variables and median for numeric variables"""
    if len(group1) > 1:
        for col in group1.columns:
            if group1[col].count() > 1:
                if np.issubdtype(group1[col], np.number):
                    group1[col] = group1[col].median()
                else:
                    group1[col] = group1[col].value_counts().index[0]
    return group1

def groupby_vehicle(data):
    """Groups by vin and summarizes categorical and numberic variables that are considered static vehicle attributes"""
    static_cols = ['vin',
                   'year', 
                   'make', 
                   'model', 
                   'trim', 
                   'body_style', 
                   'transmission', 
                   'fuel_economy_city', 
                   'fuel_economy_highway', 
                   'exterior_color']
    df_out = data.groupby('vin')[static_cols].apply(summarize_vin).drop_duplicates()
    return df_out

def groupby_listing(data):
    """Groups by vin and listing_date_begin. 
    Summarizes categorical and numberic variables that are considered static vehicle attributes"""
    dynamic_cols = ['vin', 
                'listing_date_begin',
                'mileage', 
                'accident_count',
                'is_cpo',
                'seller_city', 
                'seller_state', 
                'seller_type',
                'seller_is_franchise_dealer', 
                'seller_is_online_only',
                'seller_ships_nationwide', 
                'msrp',
                'list_price']
    df_out = data.groupby(['vin', 'listing_date_begin'])[dynamic_cols].apply(summarize_vin).drop_duplicates()
    return df_out
    
def summarize_obs(data):
    df_vehicle = groupby_vehicle(data)
    df_listing = groupby_listing(data)
    df_out = df_listing.join(df_vehicle.set_index("vin"), on="vin")
    return df_out
    
sample = df_train.head(1000)
df_tidy = summarize_obs(sample)

print("Columns: n/{}".format(df_tidy.columns))
print("Length: {}".format(len(df_tidy)))
print(df_tidy.sort_values(["vin", "listing_date_begin"]).head(100))


Columns: n/Index(['vin', 'listing_date_begin', 'mileage', 'accident_count', 'is_cpo',
       'seller_city', 'seller_state', 'seller_type',
       'seller_is_franchise_dealer', 'seller_is_online_only',
       'seller_ships_nationwide', 'msrp', 'list_price', 'year', 'make',
       'model', 'trim', 'body_style', 'transmission', 'fuel_economy_city',
       'fuel_economy_highway', 'exterior_color'],
      dtype='object')
Length: 893
                   vin listing_date_begin  mileage  accident_count is_cpo  \
288  19XFC1F30LE004389         2021-04-23   6874.0             0.0   True   
173  19XFC1F30LE004859         2021-04-01   5601.0             0.0   True   
0    19XFC1F30LE007261         2021-02-10   1924.0             0.0    NaN   
810  19XFC1F30LE202454         2020-07-14   1500.0             0.0   True   
608  19XFC1F30LE205208         2021-03-27  18500.0             0.0    NaN   
..                 ...                ...      ...             ...    ...   
315  19XFC1F38LE203285       

In [8]:
# Impute missing values

def impute_variables(data):
    """
    Use dataset values for imputation of missing variables
    """
    return data.fillna(value=imputation_dict).replace({True:1, False:0})

df_impute = impute_variables(df_tidy)

print("Columns: n/{}".format(df_impute.columns))
print("Length: {}".format(len(df_impute)))
print(df_impute.sort_values(["vin", "listing_date_begin"]).head(100))

Columns: n/Index(['vin', 'listing_date_begin', 'mileage', 'accident_count', 'is_cpo',
       'seller_city', 'seller_state', 'seller_type',
       'seller_is_franchise_dealer', 'seller_is_online_only',
       'seller_ships_nationwide', 'msrp', 'list_price', 'year', 'make',
       'model', 'trim', 'body_style', 'transmission', 'fuel_economy_city',
       'fuel_economy_highway', 'exterior_color'],
      dtype='object')
Length: 893
                   vin listing_date_begin  mileage  accident_count  is_cpo  \
288  19XFC1F30LE004389         2021-04-23   6874.0             0.0       1   
173  19XFC1F30LE004859         2021-04-01   5601.0             0.0       1   
0    19XFC1F30LE007261         2021-02-10   1924.0             0.0       0   
810  19XFC1F30LE202454         2020-07-14   1500.0             0.0       1   
608  19XFC1F30LE205208         2021-03-27  18500.0             0.0       0   
..                 ...                ...      ...             ...     ...   
315  19XFC1F38LE203285

In [26]:
# Date Handling

def handle_date(data):
    #Extracting Yea
    data['listing_year'] = pd.to_datetime(data['listing_date_begin']).dt.year
    #Extracting Month
    map_month = {1:'Jan', 2:'Feb', 3:'Mar', 
                 4:'Apr', 5:'May', 6:'Jun', 
                 7:'Jul', 8:'Aug', 9:'Sep', 
                 10:'Oct', 11:'Nov', 12:'Dec'}
    data['listing_month'] = pd.to_datetime(data['listing_date_begin']).dt.month.replace(map_month)
    return data

df_date = handle_date(df_impute)

print("Columns: n/{}".format(df_date.columns))
print("Length: {}".format(len(df_date)))
print(df_date.sort_values(["vin", "listing_date_begin"]).head(100))

Columns: n/Index(['vin', 'listing_date_begin', 'mileage', 'accident_count', 'is_cpo',
       'seller_city', 'seller_state', 'seller_type',
       'seller_is_franchise_dealer', 'seller_is_online_only',
       'seller_ships_nationwide', 'msrp', 'list_price', 'year', 'make',
       'model', 'trim', 'body_style', 'transmission', 'fuel_economy_city',
       'fuel_economy_highway', 'exterior_color', 'listing_year',
       'listing_month'],
      dtype='object')
Length: 893
                   vin listing_date_begin  mileage  accident_count  is_cpo  \
288  19XFC1F30LE004389         2021-04-23   6874.0             0.0       1   
173  19XFC1F30LE004859         2021-04-01   5601.0             0.0       1   
0    19XFC1F30LE007261         2021-02-10   1924.0             0.0       0   
810  19XFC1F30LE202454         2020-07-14   1500.0             0.0       1   
608  19XFC1F30LE205208         2021-03-27  18500.0             0.0       0   
..                 ...                ...      ...          

In [27]:
# Split feature with where it makes sense
df_date['seller_city'] = df_date.seller_city.str.split(",").str[0]

In [28]:
# Handling Outliers

def handle_outliers(data):
    numeric_cols = ['mileage', 'accident_count',
       'fuel_economy_city', 'fuel_economy_highway', 'msrp']
    print(data.describe())
    for col in numeric_cols:
        upper_lim = data[col].quantile(.95)
        lower_lim = data[col].quantile(.05)
        data_out = data[(data[col] < upper_lim) & (data[col] > lower_lim)]
    return data_out

df_outlier = handle_outliers(df_date)

print("Columns: n/{}".format(df_outlier.columns))
print("Length: {}".format(len(df_outlier)))
print(df_outlier.describe())
print(df_outlier.sort_values(["vin", "listing_date_begin"]).head(100))

            mileage  accident_count      is_cpo  seller_is_franchise_dealer  \
count    893.000000      893.000000  893.000000                  893.000000   
mean    8030.301792        0.038074    0.294513                    0.839866   
std     7038.337654        0.191482    0.456079                    0.366936   
min       56.000000        0.000000    0.000000                    0.000000   
25%     2754.000000        0.000000    0.000000                    1.000000   
50%     6273.000000        0.000000    0.000000                    1.000000   
75%    11473.000000        0.000000    1.000000                    1.000000   
max    66099.500000        1.000000    1.000000                    1.000000   

       seller_is_online_only  seller_ships_nationwide           msrp  \
count             893.000000               893.000000     893.000000   
mean                0.966405                 0.095185   33481.736517   
std                 0.180284                 0.293634   10610.430343   


In [29]:
# Normalization
def normalize(data):
    numeric_cols = ['mileage', 'accident_count','fuel_economy_city', 'fuel_economy_highway', 'msrp']
    for col in numeric_cols:
        data[col+'_normalized'] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
    return data

df_norm = normalize(df_outlier)

print("Columns: n/{}".format(df_norm.columns))
print("Length: {}".format(len(df_norm)))
print(df_norm.describe())
print(df_norm.sort_values(["vin", "listing_date_begin"]).head(100))

Columns: n/Index(['index_pandas', 'vin', 'days_on_market', 'listing_date_begin',
       'listing_date_end', 'year', 'make', 'model', 'trim', 'body_style',
       'transmission', 'mileage', 'accident_count', 'fuel_economy_city',
       'fuel_economy_highway', 'exterior_color', 'msrp', 'is_cpo',
       'seller_city', 'seller_state', 'seller_type',
       'seller_is_franchise_dealer', 'seller_is_online_only',
       'seller_ships_nationwide', 'list_price', 'shift_buyer_region',
       'mileage_normalized', 'accident_count_normalized',
       'fuel_economy_city_normalized', 'fuel_economy_highway_normalized',
       'msrp_normalized'],
      dtype='object')
Length: 696656
        index_pandas  days_on_market           year        mileage  \
count  696656.000000   696656.000000  696656.000000  668588.000000   
mean   348327.500000       52.542325    2020.087838   14139.197042   
std    201107.408904       65.811119       0.283242   11786.003820   
min         0.000000        0.000000    2020

In [ ]:
# One hot encoding
def onehot(data):
    onehot_cols = ['year', 'make', 'model', 'body_style', 'seller_city', 
                   'seller_state', 'seller_type', 'listing_year', 'listing_month']
    df_out = data
    for col in onehot_cols:
        encoded_columns = pd.get_dummies(data[col])
        df_out = df_out.join(encoded_columns).drop(col, axis=1)
    return df_out

df_onehot = onehot(df_norm)
df_onehot.columns

In [ ]:
# Make, model, body_style price

In [ ]:
# Text feature extraction

In [ ]:
df_train = df.drop(['vin','listing_date_begin', 'trim', 'transmission', 'exterior_color'])
df_test = df.drop(['vin','listing_date_begin', 'trim', 'transmission', 'exterior_color'])